In [1]:
import serial
from serial.tools import list_ports
from time import sleep

ports = list_ports.comports()
possible_devices = list()

print("Available COMs:")
i = 0
for p in ports:
    if "CH340" in p.description:
        possible_devices.append(p.device)
        print("%i >" %i, end='')
        i += 1
    print("\t%-5s: %-30s [%s]" %(p.device, p.description, p.hwid))

if len(possible_devices) == 0:
    print("No usable device found")
    i = None
elif len(possible_devices) > 1:
    print("tpye Nr. to use:")
    i = int(input())
else:
    i = 0
    
if i is not None:
    print("using %s" % possible_devices[i])
    s = serial.Serial(possible_devices[i], 115200, timeout=5)

Available COMs:
	COM1 : Kommunikationsanschluss (COM1) [ACPI\PNP0501\1]
0 >	COM5 : USB-SERIAL CH340 (COM5)        [USB VID:PID=1A86:7523 SER= LOCATION=1-1.2]
using COM5


SerialException: could not open port 'COM5': PermissionError(13, 'Access is denied.', None, 5)

In [22]:
s.flushInput()
s.flushOutput()
s.write(b"restart\n")
sleep(1)
print(s.read_all().decode())
s.write(b"print_mode -m RAW\n")
sleep(0.1)
print(s.read_all().decode())

restart
Restarting...
ets Jun  8 2016 00:22:57

rst:0xc (SW_CPU_RESET),boot:0x13 (SPI_FAST_FLASH_BOOT)
configsip: 0, SPIWP:0xee
clk_drv:0x00,q_drv:0x00,d_drv:0x00,cs0_drv:0x00,hd_drv:0x00,wp_drv:0x00
mode:DIO, clock div:1
load:0x3fff0030,len:6940
ho 0 tail 12 room 4
load:0x40078000,len:15500
load:0x40080400,len:3844
entry 0x4008064c
I (29) boot: ESP-IDF 5.0.1 2nd stage bootloader
I (29) boot: compile time 10:44:23
I (29) boot: chip revision: v1.0
I (32) boot.esp32: SPI Speed      : 80MHz
I (37) boot.esp32: SPI Mode       : DIO
I (41) boot.esp32: SPI Flash Size : 4MB
I (46) boot: Enabling RNG early entropy source...
I (51) boot: Partition Table:
I (55) boot: ## Label            Usage          Type ST Offset   Length
I (62) boot:  0 nvs              WiFi data        01 02 00009000 00006000
I (69) boot:  1 phy_init         RF data          01 01 0000f000 00001000
I (77) boot:  2 factory          factory app      00 00 00010000 00100000
I (84) boot: End of partition table
I (88) esp_image:

In [3]:
s.write(b"adc_read -r 7 -l 16\n")
sleep(0.5)
print(s.read_all().decode())

adc_read -r 7 -l 16
r[CDE#10]
esp> 


In [4]:
s.write(b"pmu_rPMU -c 1\n")
sleep(0.5)
print(s.read_all().decode())

pmu_rPMU -c 1
t[11000000#1D]
r[1E060#18]
esp> 


In [5]:
s.write(b"pmu_reset\n")
s.write(b"mem_read -a 0 -l 8\n")
sleep(0.5)
print(s.read_all().decode())
s.write(b"pmu_release\n")
sleep(0.2)
print(s.read_all().decode())

pmu_reset
Setting reset for PMU
esp> mem_read -a 0 -l 8
r[55#8]
esp> 
pmu_release
Releasing reset for PMU
esp> 


In [12]:
adr = 0

s.flushInput()
s.write(b"pmu_reset\n")
s.write(b"mem_read -a 0 -l 16\n")

sleep(0.2)

s.read_until(b"r[")
raw_data_str = s.read_until(b"]")[:-1].decode()

print(raw_data_str)

data, length = raw_data_str.split("#")

data = int(data, 16)
length = int(length, 16)

print(data)
print(length)

5501#10
21761
16


In [33]:
s.close()

In [1]:
from pmu_som import communicator

c = communicator.communicator(True)

Available COMs:
	COM1 : Kommunikationsanschluss (COM1) [ACPI\PNP0501\0]
0 >	COM28: USB-SERIAL CH340 (COM28)       [USB VID:PID=1A86:7523 SER=6 LOCATION=1-7.4]
restart
Restarting...
ets Jun  8 2016 00:22:57

rst:0xc (SW_CPU_RESET),boot:0x13 (SPI_FAST_FLASH_BOOT)
configsip: 0, SPIWP:0xee
clk_drv:0x00,q_drv:0x00,d_drv:0x00,cs0_drv:0x00,hd_drv:0x00,wp_drv:0x00
mode:DIO, clock div:1
load:0x3fff0030,len:6940
ho 0 tail 12 room 4
load:0x40078000,len:15500
load:0x40080400,len:3844
entry 0x4008064c
I (29) boot: ESP-IDF 5.0.1 2nd stage bootloader
I (29) boot: compile time 10:44:23
I (29) boot: chip revision: v1.0
I (32) boot.esp32: SPI Speed      : 80MHz
I (37) boot.esp32: SPI Mode       : DIO
I (41) boot.esp32: SPI Flash Size : 4MB
I (46) boot: Enabling RNG early entropy source...
I (51) boot: Partition Table:
I (55) boot: ## Label            Usage          Type ST Offset   Length
I (62) boot:  0 nvs              WiFi data        01 02 00009000 00006000
I (69) boot:  1 phy_init         RF data  

In [2]:
c.command("adc_read -r 7 -l 16")

CDE#10]


3294

In [3]:
del c

In [7]:
test = [0,1,2,3,4,5,6,7,8,9]
t_sl = slice(0,10)

test[t_sl]

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]